In [1]:
#For Kaggle
#date=''

#For WL data
source='WL'
date='20241118'

saveImage=False

packageName='06-sklearn.tree'
classifierName='DecisionTreeClassifier'
extraParameterName='7-SmoteOverSampling'

print('done')

done


In [2]:
import sys
import os
from importlib import reload
fpath = os.path.join('..//scripts')
sys.path.append(fpath)

import warnings
warnings.filterwarnings('ignore')

#loading internal scripts
import frauddetection as fd
import sourcedata as sd
import dataimport as di
import result as resultMd
import sampling as sp
reload(fd)
reload(sd)
reload(di)
reload(resultMd)
reload(sp)

print('done')

done


In [3]:
# Hyperparameters Results
# not too high to avoid overfitting (lower or equals to 12 ?)
max_depthFound=23
min_samples_leafFound=2
min_samples_splitFound=10
criterionFound='entropy'



resultMd.update_hyperparameter_config_result(packageName,classifierName,extraParameterName,max_depthFound)


print('done')

done


In [4]:
import pandas as pd

dfTrx0 = pd.read_csv('../data/cleaned/'+source+'export'+date+'.csv')
predictors = fd.getPredictors(dfTrx0)
dfTrx=fd.getStandardScaledData('export'+date+'.csv',source,predictors)
dfTrx.head(5)

,mcd_fraud_score,vaa_score,TRX_3D_SECURED,trx_accepted,trx_cnp,ecom_indicator_group,ecom,trx_authentication_group,ch_present_group,trx_response_code_group,...,clusterCardHolder_cluster4,clusterMerchant_M0,clusterMerchant_M1,clusterMerchant_M2,clusterMerchant_M3,nbPreviousTrx,previousTrxAmountSumLog,trx_amount_log10,distancePrevTrx,Class
0,5.047982e-01,9.678365e-16,-0.316345,0.410697,0.895963,1.188319,0.656144,-0.640003,0.725223,-0.119809,...,-0.538119,1.055837,-0.358732,-0.224417,-0.721017,-0.445922,-0.731315,0.639959,0.409416,0
1,-6.112106e-01,9.678365e-16,3.161103,0.410697,0.895963,-0.841525,0.656144,1.562492,0.725223,-0.119809,...,-0.538119,-0.947116,-0.358732,-0.224417,1.386930,-0.445922,-0.731315,0.028804,-1.283070,0
2,1.090505e+00,9.678365e-16,-0.316345,0.410697,0.895963,1.188319,0.656144,-0.640003,0.725223,-0.119809,...,-0.538119,1.055837,-0.358732,-0.224417,-0.721017,-0.445922,-0.731315,-0.575760,0.409416,0
3,-6.748697e-16,9.678365e-16,-0.316345,0.410697,-1.116118,-0.841525,-1.524055,1.562492,-1.378886,-0.119809,...,-0.538119,-0.947116,-0.358732,-0.224417,-0.721017,-0.445922,-0.731315,0.358182,-0.100479,0
4,-6.748697e-16,5.096885e-01,-0.316345,0.410697,-1.116118,-0.841525,0.656144,-0.640003,0.725223,-0.119809,...,1.858325,1.055837,-0.358732,-0.224417,-0.721017,-0.445922,-0.731315,0.333060,0.409416,0


In [5]:
from sklearn.model_selection import train_test_split

predictors = fd.getPredictors(dfTrx0)
TEST_SIZE = 0.20 # test size using_train_test_split
RANDOM_STATE = 0
x_train0, x_test, y_train0, y_test = train_test_split(dfTrx[predictors], dfTrx['Class'], test_size = TEST_SIZE, 
                                                        stratify= dfTrx['Class'],
                                                        random_state = RANDOM_STATE)

x_train, y_train = sp.smoteOverSampling(x_train0, y_train0,rateOverSampling=5)


Before Sampling shape (847879, 39) fraud rate 0.00123 
Duration 1.2 s 
After Sampling shape  (852039, 39) fraud rate 0.00614 


In [6]:
%%script false
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import randint

modelClf = DecisionTreeClassifier(random_state=42)
dic_param={
    'criterion':["gini","entropy"],
    'max_depth': randint(7,16),
    'min_samples_leaf': randint(10,25),
    'min_samples_split': randint(15,25),
}

res=fd.hyperparameterSelectionRandomizedSearchCVSampling(modelClf, dic_param, 'f1', x_train, y_train,iter=5)
print(res)


#{'criterion': 'entropy', 'max_depth': 14, 'min_samples_leaf': 10, 'min_samples_split': 22}
#0.6894583240663005
#score   0.7674570371981727



Couldn't find program: 'false'


In [ ]:
#%%script false
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import randint

modelClf = DecisionTreeClassifier(random_state=42)

dic_param={
    'criterion':["gini","entropy"],
    'min_samples_leaf': [1,2,3],
    'min_samples_split': [2,6,8,10,12],
    'max_depth': [22,23,24,25]
}


res=fd.hyperparameterSelectionGridSearchCVSampling(modelClf, dic_param, 'f1', x_train, y_train)
print(res)


#{'criterion': 'entropy', 'max_depth': 21, 'min_samples_leaf': 2, 'min_samples_split': 16}
#0.7433509007491967
#scoref1 0.8757073565076798

#{'criterion': 'entropy', 'max_depth': 22, 'min_samples_leaf': 2, 'min_samples_split': 13}
#0.7511668851596761
#scoref1 0.8864227154569086

#{'criterion': 'entropy', 'max_depth': 23, 'min_samples_leaf': 2, 'min_samples_split': 10}
#0.7469044952691073
#scoref1 0.9071399919775371



Fitting 5 folds for each of 120 candidates, totalling 600 fits
[CV 1/5; 1/120] START criterion=gini, max_depth=22, min_samples_leaf=1, min_samples_split=2
[CV 1/5; 1/120] END criterion=gini, max_depth=22, min_samples_leaf=1, min_samples_split=2;, score=0.519 total time=  11.5s
[CV 2/5; 1/120] START criterion=gini, max_depth=22, min_samples_leaf=1, min_samples_split=2
[CV 2/5; 1/120] END criterion=gini, max_depth=22, min_samples_leaf=1, min_samples_split=2;, score=0.807 total time=  13.2s
[CV 3/5; 1/120] START criterion=gini, max_depth=22, min_samples_leaf=1, min_samples_split=2
[CV 3/5; 1/120] END criterion=gini, max_depth=22, min_samples_leaf=1, min_samples_split=2;, score=0.786 total time=  14.2s
[CV 4/5; 1/120] START criterion=gini, max_depth=22, min_samples_leaf=1, min_samples_split=2
[CV 4/5; 1/120] END criterion=gini, max_depth=22, min_samples_leaf=1, min_samples_split=2;, score=0.786 total time=  13.4s
[CV 5/5; 1/120] START criterion=gini, max_depth=22, min_samples_leaf=1, min_s

In [ ]:
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
from datetime import datetime

then= datetime.now()
predictors = fd.getPredictors(dfTrx0)


modelClf = DecisionTreeClassifier(random_state=42)
parameters={'criterion': criterionFound, 'max_depth': max_depthFound, 
            'min_samples_leaf': min_samples_leafFound, 'min_samples_split': min_samples_splitFound}
modelClf.set_params(**parameters)

modelClf.fit(x_train, y_train)
predsTrain = modelClf.predict(x_train)
predsTest = modelClf.predict(x_test)

now = datetime.now()
duration= now - then
duration_in_s = duration.total_seconds()
print("Duration ",duration_in_s)
f1,mcc,roc=fd.print_scores(y_test, predsTest,'all')

f1Train=fd.print_scores(y_train, predsTrain,'f1', False)
f1Test=fd.print_scores(y_test, predsTest,'f1')
fd.show_importance(modelClf, predictors)
fd.show_confusion_matrix(y_test, predsTest)
fd.show_prediction_graph(modelClf, x_test,y_test,'../imgs/FD06A-DecisionTreeClassifierProbaHistogram' if saveImage else None)

diffF1=f1Train-f1Test
print('diffF1',diffF1)

In [ ]:
files = fd.getAllFiles()
predictors = fd.getPredictors(dfTrx0)

range = []
f1s = []
rocs = []
loop =0
for file in files:
    loop=loop+1
    range.append(loop)
    print(file)
  
    dfTrx=fd.getStandardScaledData(file,source,predictors)
    preds = modelClf.predict(dfTrx[predictors])

    f1,mcc,roc= fd.print_scores(dfTrx['Class'], preds,'All', True)
    #fd.show_importance(modelClf,predictors)
    fd.show_confusion_matrix(dfTrx['Class'], preds)
    f1s.append(f1)
    rocs.append(roc)

fd.plt_train_test(range, f1s)
resultMd.update_performance_nextdays_result(packageName,classifierName,extraParameterName, f1s[0],f1s[1],f1s[2],f1s[3],rocs[0],rocs[1],rocs[2],rocs[3],diffF1)

